# Lab 1: LangChain Agents

**Objective:** Build a research assistant agent with custom tools, memory, and debugging capabilities.

**Duration:** ~45 minutes

**What You'll Learn:**
- How to create custom tools using the `@tool` decorator
- How to build agents with the ReAct pattern
- How to add conversational memory
- How to debug agent behavior

## Part 1: Setup and Imports

First, let's import the required libraries and configure our environment.

In [1]:
# Environment setup
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API keys
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found in environment"
print("Environment configured successfully!")

Environment configured successfully!


In [2]:
# Core imports
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain import hub
from langchain.memory import ConversationBufferMemory

print("Imports successful!")

Imports successful!


## Part 2: Understanding Tools

Tools are functions that agents can call to interact with the outside world. In LangChain, we use the `@tool` decorator to define them.

In [3]:
# Simple calculator tool
@tool
def calculator(expression: str) -> str:
    """Evaluates a mathematical expression and returns the result.
    Use this for any math calculations.
    
    Args:
        expression: A mathematical expression like '2 + 2' or '15 * 7'
    """
    try:
        # Only allow safe mathematical operations
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"

# Test the tool
print(calculator.invoke("15 * 7 + 3"))

The result of 15 * 7 + 3 is 108


In [4]:
# Current date/time tool
from datetime import datetime

@tool
def get_current_datetime() -> str:
    """Returns the current date and time. Use this when you need to know what time or date it is."""
    now = datetime.now()
    return f"Current date and time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

# Test the tool
print(get_current_datetime.invoke(""))

Current date and time: 2025-12-05 10:07:35


In [5]:
# Web search tool using Tavily
from langchain_community.tools.tavily_search import TavilySearchResults

# Check if Tavily API key is available
if os.getenv("TAVILY_API_KEY"):
    search_tool = TavilySearchResults(
        max_results=3,
        description="Search the web for current information. Use this when you need up-to-date information."
    )
    print("Tavily search tool configured!")
else:
    # Fallback mock search for demo purposes
    @tool
    def search_tool(query: str) -> str:
        """Search the web for information (mock version)."""
        return f"Mock search results for: {query}. In production, configure TAVILY_API_KEY for real search."
    print("Using mock search tool (set TAVILY_API_KEY for real search)")

Tavily search tool configured!


/var/folders/gk/hk1464k11yg67s5xbrchbgsm0000gn/T/ipykernel_864/985916128.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(


## Part 3: Building Your First Agent

Now let's build an agent that can use these tools.

In [6]:
# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0  # Lower temperature for more consistent behavior
)

# Define our tools list
tools = [calculator, get_current_datetime, search_tool]

print(f"Configured {len(tools)} tools:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description[:50]}...")

Configured 3 tools:
  - calculator: Evaluates a mathematical expression and returns th...
  - get_current_datetime: Returns the current date and time. Use this when y...
  - tavily_search_results_json: Search the web for current information. Use this w...


In [7]:
# Get the ReAct prompt from LangChain hub
prompt = hub.pull("hwchase17/react")

# Let's examine the prompt structure
print("ReAct Prompt Template:")
print("=" * 50)
print(prompt.template[:500] + "...")

ReAct Prompt Template:
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the...


In [8]:
# Create the ReAct agent
agent = create_react_agent(llm, tools, prompt)

# Wrap in an executor with verbose mode for debugging
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Shows the agent's thought process
    max_iterations=5,  # Prevent infinite loops
    handle_parsing_errors=True
)

print("Agent created successfully!")

Agent created successfully!


In [9]:
# Test the agent with a simple query
response = agent_executor.invoke({
    "input": "What is 15 multiplied by 7, and what time is it right now?"
})

print("\n" + "=" * 50)
print("Final Answer:", response["output"])



> Entering new AgentExecutor chain...
I need to calculate the product of 15 and 7, and then find out the current date and time. 

Action: calculator  
Action Input: '15 * 7'  Error: Invalid characters in expressionIt seems there was an error in the calculation attempt. I will try again to calculate the product of 15 and 7. 

Action: calculator  
Action Input: '15 * 7'  Error: Invalid characters in expressionIt seems there is a persistent issue with the calculator function. I will try a different approach to calculate the product of 15 and 7. 

Action: calculator  
Action Input: '15 * 7'  Error: Invalid characters in expressionIt appears that the calculator function is not working as expected. I will proceed to find the current date and time first, and then I will calculate the product of 15 and 7 manually.

Action: get_current_datetime  
Action Input: ''  Current date and time: 2025-12-05 10:07:55I have the current date and time. Now, I will calculate the product of 15 and 7 manually

### Understanding the Agent's Reasoning

Notice how the agent:
1. **Thought** about what tools it needs
2. **Acted** by calling the appropriate tool
3. **Observed** the result
4. **Repeated** until it had all the information
5. **Responded** with the final answer

This is the **ReAct** (Reason + Act) pattern in action!

## Part 4: Adding Custom Tools

Let's create more sophisticated custom tools.

In [11]:
# Weather tool (simulated)
from langchain.tools import tool

@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city.
    
    Args:
        city: The name of the city to get weather for
    """
    # In production, this would call a real weather API
    import random
    conditions = ["sunny", "cloudy", "rainy", "partly cloudy"]
    temp = random.randint(50, 85)
    condition = random.choice(conditions)
    return f"Weather in {city}: {temp}°F, {condition}"

# Test it
print(get_weather.invoke("San Francisco"))

Weather in San Francisco: 52°F, partly cloudy


In [12]:
# Note-taking tool with state
notes_storage = []

@tool
def save_note(note: str) -> str:
    """Save a note for later reference.
    
    Args:
        note: The note content to save
    """
    notes_storage.append(note)
    return f"Note saved! You now have {len(notes_storage)} note(s)."

@tool
def get_notes() -> str:
    """Retrieve all saved notes."""
    if not notes_storage:
        return "No notes saved yet."
    return "Saved notes:\n" + "\n".join(f"- {note}" for note in notes_storage)

# Test
print(save_note.invoke("Remember to review LangChain docs"))
print(get_notes.invoke(""))

Note saved! You now have 1 note(s).
Saved notes:
- Remember to review LangChain docs


In [13]:
# Create an enhanced agent with all tools
enhanced_tools = [calculator, get_current_datetime, search_tool, get_weather, save_note, get_notes]

enhanced_agent = create_react_agent(llm, enhanced_tools, prompt)

enhanced_executor = AgentExecutor(
    agent=enhanced_agent,
    tools=enhanced_tools,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

print(f"Enhanced agent with {len(enhanced_tools)} tools ready!")

Enhanced agent with 6 tools ready!


In [14]:
# Test the enhanced agent
response = enhanced_executor.invoke({
    "input": "What's the weather in New York? Save a note about it."
})

print("\n" + "=" * 50)
print("Final Answer:", response["output"])



> Entering new AgentExecutor chain...
I need to check the current weather in New York and then save that information as a note.  
Action: get_weather  
Action Input: "New York"  Weather in New York: 68°F, sunnyI have the current weather information for New York. Now, I will save this information as a note.  
Action: save_note  
Action Input: "Current weather in New York: 68°F, sunny"  Note saved! You now have 2 note(s).I now know the final answer.  
Final Answer: The current weather in New York is 68°F and sunny, and this information has been saved as a note.

> Finished chain.

Final Answer: The current weather in New York is 68°F and sunny, and this information has been saved as a note.


## Part 5: Adding Memory

Agents become more useful when they can remember previous interactions.

In [15]:
# Get the conversational ReAct prompt
conversational_prompt = hub.pull("hwchase17/react-chat")

print("Conversational prompt loaded!")
print("\nInput variables:", conversational_prompt.input_variables)

Conversational prompt loaded!

Input variables: ['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools']


In [16]:
# Create memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create conversational agent
conversational_agent = create_react_agent(llm, enhanced_tools, conversational_prompt)

conversational_executor = AgentExecutor(
    agent=conversational_agent,
    tools=enhanced_tools,
    memory=memory,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

print("Conversational agent with memory ready!")

Conversational agent with memory ready!


/var/folders/gk/hk1464k11yg67s5xbrchbgsm0000gn/T/ipykernel_864/892689015.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [17]:
# First interaction
response1 = conversational_executor.invoke({
    "input": "My name is Alice. What's the weather in Seattle?"
})
print("\nResponse 1:", response1["output"])



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: get_weather
Action Input: Seattle
```Weather in Seattle
```: 51°F, rainyDo I need to use a tool? No
Final Answer: The current weather in Seattle is 51°F and rainy.

> Finished chain.

Response 1: The current weather in Seattle is 51°F and rainy.


In [18]:
# Second interaction - the agent should remember the name
response2 = conversational_executor.invoke({
    "input": "What's my name? And save a note that says I'm interested in Seattle weather."
})
print("\nResponse 2:", response2["output"])



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: save_note
Action Input: I'm interested in Seattle weather.Note saved! You now have 3 note(s).Do I need to use a tool? No  
Final Answer: Your name is Alice. I have also saved a note that says you're interested in Seattle weather.

> Finished chain.

Response 2: Your name is Alice. I have also saved a note that says you're interested in Seattle weather.


In [19]:
# Third interaction - check memory and notes
response3 = conversational_executor.invoke({
    "input": "What notes do we have? Summarize our conversation."
})
print("\nResponse 3:", response3["output"])



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: get_notes
Action Input: 
```Saved notes:
- Remember to review LangChain docs
- Current weather in New York: 68°F, sunny
- I'm interested in Seattle weather.Do I need to use a tool? No  
Final Answer: You have the following notes saved:  
1. Remember to review LangChain docs.  
2. Current weather in New York: 68°F, sunny.  
3. I'm interested in Seattle weather.  

To summarize our conversation: You asked about the weather in Seattle, and I informed you that it was 51°F and rainy. You then mentioned your name is Alice and requested to save a note about your interest in Seattle weather.

> Finished chain.

Response 3: You have the following notes saved:  
1. Remember to review LangChain docs.  
2. Current weather in New York: 68°F, sunny.  
3. I'm interested in Seattle weather.  

To summarize our conversation: You asked about the weather in Seattle, and I informed you that it was 51°F and rainy. You

## Part 6: Debugging and Best Practices

Let's explore techniques for debugging and improving agent behavior.

In [20]:
# Enable LangChain debugging
from langchain.globals import set_debug, set_verbose

# Uncomment to enable detailed debugging
# set_debug(True)
set_verbose(True)

print("Debug settings configured")

Debug settings configured


In [21]:
# Custom error handling tool
@tool
def risky_operation(data: str) -> str:
    """A tool that might fail - for demonstrating error handling.
    
    Args:
        data: Input data to process
    """
    if "error" in data.lower():
        raise ValueError("Simulated error for demonstration")
    return f"Successfully processed: {data}"

# The agent with handle_parsing_errors=True will gracefully handle this
print("Risky operation tool created")

Risky operation tool created


In [22]:
# Best practice: Create well-documented tools
@tool
def analyze_sentiment(text: str) -> str:
    """Analyze the sentiment of a given text.
    
    This tool examines text and determines if the overall sentiment
    is positive, negative, or neutral. Use this when you need to
    understand how someone feels about something based on their writing.
    
    Args:
        text: The text to analyze for sentiment
        
    Returns:
        A string describing the sentiment (positive, negative, or neutral)
        along with confidence indicators.
    """
    # Simplified sentiment analysis
    positive_words = ['good', 'great', 'excellent', 'happy', 'love', 'amazing', 'wonderful']
    negative_words = ['bad', 'terrible', 'awful', 'sad', 'hate', 'horrible', 'disappointing']
    
    text_lower = text.lower()
    pos_count = sum(1 for word in positive_words if word in text_lower)
    neg_count = sum(1 for word in negative_words if word in text_lower)
    
    if pos_count > neg_count:
        return f"Sentiment: POSITIVE (found {pos_count} positive indicators)"
    elif neg_count > pos_count:
        return f"Sentiment: NEGATIVE (found {neg_count} negative indicators)"
    else:
        return "Sentiment: NEUTRAL (no strong indicators found)"

# Test
print(analyze_sentiment.invoke("I love this amazing product! It's wonderful."))

Sentiment: POSITIVE (found 3 positive indicators)


## Part 7: Payments Industry Example - Fraud Detection Tools

Let's build a set of tools that would be useful in a payments/financial services context. This demonstrates how agents can be applied to real-world business problems.

In [ ]:
# Simulated transaction database for payments examples
TRANSACTION_DB = {
    "TX-001": {"amount": 150.00, "merchant": "Coffee Shop", "category": "Food", "location": "US", "time": "09:15", "status": "completed"},
    "TX-002": {"amount": 5000.00, "merchant": "Electronics Plus", "category": "Electronics", "location": "Nigeria", "time": "03:22", "status": "pending"},
    "TX-003": {"amount": 89.99, "merchant": "Gas Station", "category": "Gas", "location": "US", "time": "17:45", "status": "completed"},
    "TX-004": {"amount": 2500.00, "merchant": "Luxury Goods Int'l", "category": "Luxury", "location": "Russia", "time": "02:11", "status": "pending"},
    "TX-005": {"amount": 45.00, "merchant": "Grocery Mart", "category": "Grocery", "location": "US", "time": "14:30", "status": "completed"},
}

CUSTOMER_HISTORY = {
    "avg_transaction": 125.00,
    "typical_locations": ["US", "Canada"],
    "typical_categories": ["Food", "Gas", "Grocery"],
    "typical_hours": range(7, 23),  # 7 AM to 11 PM
}

# Tool: Get transaction details
@tool
def get_transaction(transaction_id: str) -> str:
    """Retrieve details for a specific transaction.
    
    Use this to look up transaction information by ID.
    
    Args:
        transaction_id: The transaction ID (e.g., TX-001)
        
    Returns:
        Transaction details including amount, merchant, location, and status
    """
    if transaction_id in TRANSACTION_DB:
        txn = TRANSACTION_DB[transaction_id]
        return f"""Transaction {transaction_id}:
  Amount: ${txn['amount']:.2f}
  Merchant: {txn['merchant']}
  Category: {txn['category']}
  Location: {txn['location']}
  Time: {txn['time']}
  Status: {txn['status']}"""
    return f"Transaction {transaction_id} not found"

# Tool: Get customer spending history
@tool
def get_customer_history() -> str:
    """Get the customer's typical spending patterns and history.
    
    Use this to understand what's normal for this customer.
    
    Returns:
        Customer's average transaction, typical locations, and spending categories
    """
    return f"""Customer Spending Profile:
  Average Transaction: ${CUSTOMER_HISTORY['avg_transaction']:.2f}
  Typical Locations: {', '.join(CUSTOMER_HISTORY['typical_locations'])}
  Typical Categories: {', '.join(CUSTOMER_HISTORY['typical_categories'])}
  Active Hours: 7 AM - 11 PM"""

# Tool: Calculate risk score
@tool
def calculate_risk_score(transaction_id: str) -> str:
    """Calculate a fraud risk score for a transaction.
    
    Analyzes the transaction against customer patterns and returns a risk score.
    
    Args:
        transaction_id: The transaction ID to analyze
        
    Returns:
        Risk score (0-100) with explanation of risk factors
    """
    if transaction_id not in TRANSACTION_DB:
        return f"Transaction {transaction_id} not found"
    
    txn = TRANSACTION_DB[transaction_id]
    risk_score = 0
    risk_factors = []
    
    # Check amount vs average
    amount_ratio = txn['amount'] / CUSTOMER_HISTORY['avg_transaction']
    if amount_ratio > 10:
        risk_score += 30
        risk_factors.append(f"Amount is {amount_ratio:.1f}x above average")
    elif amount_ratio > 5:
        risk_score += 15
        risk_factors.append(f"Amount is {amount_ratio:.1f}x above average")
    
    # Check location
    if txn['location'] not in CUSTOMER_HISTORY['typical_locations']:
        risk_score += 25
        risk_factors.append(f"Unusual location: {txn['location']}")
    
    # Check category
    if txn['category'] not in CUSTOMER_HISTORY['typical_categories']:
        risk_score += 15
        risk_factors.append(f"Unusual category: {txn['category']}")
    
    # Check time
    hour = int(txn['time'].split(':')[0])
    if hour not in CUSTOMER_HISTORY['typical_hours']:
        risk_score += 20
        risk_factors.append(f"Unusual time: {txn['time']}")
    
    risk_level = "LOW" if risk_score < 30 else "MEDIUM" if risk_score < 60 else "HIGH"
    
    factors_str = "\n  - ".join(risk_factors) if risk_factors else "None identified"
    return f"""Risk Analysis for {transaction_id}:
  Risk Score: {risk_score}/100 ({risk_level})
  Risk Factors:
  - {factors_str}"""

# Tool: Flag transaction for review
@tool
def flag_for_review(transaction_id: str, reason: str = "") -> str:
    """Flag a transaction for manual review by the fraud team.
    
    Use this when a transaction appears suspicious and needs human review.
    
    Args:
        transaction_id: The transaction to flag
        reason: The reason for flagging (e.g., "High risk score, unusual location").
                Optional; if not provided, a default message is used.
        
    Returns:
        Confirmation that the transaction was flagged
    """
    if transaction_id not in TRANSACTION_DB:
        return f"Transaction {transaction_id} not found"

    if not reason:
        reason = "No reason provided"

    return f"""✓ Transaction {transaction_id} flagged for review
  Reason: {reason}
  Status: Added to fraud review queue (HIGH PRIORITY)
  Estimated Review Time: 2-4 hours"""

print("Payments tools created successfully!")
print("\nAvailable tools:")
print("  - get_transaction: Look up transaction details")
print("  - get_customer_history: View customer spending patterns")
print("  - calculate_risk_score: Analyze fraud risk")
print("  - flag_for_review: Flag suspicious transactions")

Payments tools created successfully!

Available tools:
  - get_transaction: Look up transaction details
  - get_customer_history: View customer spending patterns
  - calculate_risk_score: Analyze fraud risk
  - flag_for_review: Flag suspicious transactions


In [24]:
# Create a Fraud Detection Agent with the payments tools
fraud_tools = [get_transaction, get_customer_history, calculate_risk_score, flag_for_review, calculator]

fraud_agent = create_react_agent(llm, fraud_tools, prompt)

fraud_executor = AgentExecutor(
    agent=fraud_agent,
    tools=fraud_tools,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

print("Fraud Detection Agent ready!")
print(f"Tools available: {[t.name for t in fraud_tools]}")

Fraud Detection Agent ready!
Tools available: ['get_transaction', 'get_customer_history', 'calculate_risk_score', 'flag_for_review', 'calculator']


In [25]:
# Test the Fraud Detection Agent
# This query demonstrates the agent using multiple tools to analyze a suspicious transaction

response = fraud_executor.invoke({
    "input": """Analyze transaction TX-002 for potential fraud. 
    First get the transaction details, then check the customer's normal spending patterns,
    calculate the risk score, and if it's high risk, flag it for review."""
})

print("\n" + "=" * 50)
print("Final Answer:", response["output"])



> Entering new AgentExecutor chain...
I need to start by retrieving the details of transaction TX-002 to understand its specifics. 
Action: get_transaction
Action Input: "TX-002"Transaction TX-002:
  Amount: $5000.00
  Merchant: Electronics Plus
  Category: Electronics
  Location: Nigeria
  Time: 03:22
  Status: pendingThe transaction is for a significant amount of $5000 at an electronics merchant located in Nigeria, and it is currently pending. Next, I need to check the customer's normal spending patterns to see if this transaction aligns with their typical behavior. 

Action: get_customer_history  
Action Input: ""  Customer Spending Profile:
  Average Transaction: $125.00
  Typical Locations: US, Canada
  Typical Categories: Food, Gas, Grocery
  Active Hours: 7 AM - 11 PMThe customer's average transaction amount is $125, and they typically spend in the US and Canada on categories like food, gas, and grocery. The transaction in question is for $5000 at an electronics store in Niger

ValidationError: 1 validation error for flag_for_review
reason
  Field required [type=missing, input_value={'transaction_id': 'TX-00... location and category'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [ ]:
# Try analyzing a low-risk transaction for comparison
response_low_risk = fraud_executor.invoke({
    "input": "Analyze transaction TX-001. Is it suspicious?"
})

print("\n" + "=" * 50)
print("Final Answer:", response_low_risk["output"])

## Challenge: Build Your Own Research Agent

Now it's your turn! Create a research assistant agent with the following capabilities:

1. **Web search** - Find current information
2. **Note taking** - Save important findings
3. **Calculator** - Perform calculations
4. **Summary tool** - Summarize findings

The agent should be able to:
- Research a topic
- Save key findings as notes
- Perform any needed calculations
- Provide a summary

In [ ]:
# TODO: Create a summary tool
@tool
def summarize_research() -> str:
    """Summarize all research findings from saved notes.
    
    Use this after gathering information to create a summary
    of all saved notes.
    """
    # TODO: Implement this
    # Hint: Use the notes_storage list
    pass

# TODO: Create your research agent with all the tools
# research_tools = [...]
# research_agent = create_react_agent(...)
# research_executor = AgentExecutor(...)

In [ ]:
# TODO: Test your research agent
# Example query: "Research the current state of AI in healthcare. 
#                 Save the key findings and provide a summary."

## Lab Summary

In this lab, you learned:

1. **Tool Creation**: Using `@tool` decorator to create agent tools
2. **ReAct Pattern**: How agents reason and act in a loop
3. **Agent Building**: Creating agents with `create_react_agent`
4. **Memory**: Adding conversational memory with `ConversationBufferMemory`
5. **Debugging**: Using verbose mode and debugging tools

### Key Takeaways

- Good tool descriptions are **critical** - they help the agent understand when to use each tool
- Use `verbose=True` during development to understand agent behavior
- Set `max_iterations` to prevent infinite loops
- Memory enables multi-turn conversations
- Error handling (`handle_parsing_errors=True`) makes agents more robust

### Next Steps

In Lab 2, you'll learn about multi-agent systems with AutoGen and CrewAI!

In [ ]:
# Cleanup
set_verbose(False)
print("Lab 1 complete!")